In [1]:
# third party
import hagrid
import numpy as np

# syft absolute
import syft as sy
from syft.client.domain_client import DomainClient
from syft.client.gateway_client import GatewayClient

print(f"{hagrid.__version__ = }")
print(f"{sy.__version__ = }")

hagrid.__version__ = '0.3.112'
sy.__version__ = '0.8.5-beta.10'


## 0. Launching Nodes

### Docker Nodes (Deprecated)

```bash
bash -c '\
    hagrid launch test-gateway-1 gateway to docker:9081 --no-health-checks --verbose --no-warnings --build --dev
    hagrid launch test-domain-1 domain to docker:9091 --verbose --enable-signup --no-warnings --no-health-checks --build --dev
    hagrid launch test-domain-2 domain to docker:9092 --verbose --enable-signup --no-warnings --no-health-checks --build --dev'
```

### K8s Nodes


#### Deleting existing nodes
```bash
bash -c '\
    CLUSTER_NAME=testgateway1 tox -e dev.k8s.destroy || true && \
    CLUSTER_NAME=testdomain1 tox -e dev.k8s.destroy || true && \
    CLUSTER_NAME=testdomain2 tox -e dev.k8s.destroy || true'
```


#### Launch a Gateway Node

```bash
bash -c '\
    export CLUSTER_NAME=testgateway1 CLUSTER_HTTP_PORT=9081 DEVSPACE_PROFILE=gateway && \
    tox -e dev.k8s.start && \
    tox -e dev.k8s.hotreload'
```

#### Launch 2 Domain Nodes
```bash
bash -c '\
    export CLUSTER_NAME=testdomain1 CLUSTER_HTTP_PORT=9091 && \
    tox -e dev.k8s.start && \
    tox -e dev.k8s.hotreload'
```

```bash
bash -c '\
    export CLUSTER_NAME=testdomain2 CLUSTER_HTTP_PORT=9092 && \
    tox -e dev.k8s.start && \
    tox -e dev.k8s.hotreload'
```

### Web Server Nodes

In [ ]:
gateway_node = sy.Orchestra.launch(
    name="testgateway1",
    node_type=sy.NodeType.GATEWAY,
    dev_mode=True,
    reset=True,
    port=9081,
)

In [ ]:
domain_node1 = sy.Orchestra.launch(
    name="testdomain1",
    node_type=sy.NodeType.DOMAIN,
    dev_mode=True,
    reset=True,
    port=9091,
)
domain_node2 = sy.Orchestra.launch(
    name="testgateway1",
    node_type=sy.NodeType.DOMAIN,
    dev_mode=True,
    reset=True,
    port=9092,
)

In [ ]:
sy.gateways

In [ ]:
sy.gateways.all_networks

### Check nodes health

In [2]:
hagrid.check("localhost:9081", timeout=120)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:9081/login               │ ✅ │
│ api       │ http://localhost:9081/api/v2/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

In [ ]:
hagrid.check("localhost:9091", timeout=120)

In [ ]:
hagrid.check("localhost:9092", timeout=120)

### Check gateways and domains registry

In [ ]:
sy.gateways

In [ ]:
sy.domains

In [ ]:
sy.search("dataset_1")

## 1. Login to the Gateway Node

In [ ]:
gateway_client: GatewayClient = sy.login(
    port=9081, email="info@openmined.org", password="changethis"
)

In [ ]:
gateway_client.peers

## 2. Login to the domain nodes, connecting to gateways, upload a dataset, create some data scientist accounts

In [ ]:
domain_client1: DomainClient = sy.login(
    port=9091, email="info@openmined.org", password="changethis"
)
domain_client2: DomainClient = sy.login(
    port=9092, email="info@openmined.org", password="changethis"
)

In [ ]:
res = domain_client1.connect_to_gateway(gateway_client)
print(res)
res = domain_client2.connect_to_gateway(gateway_client)
res

In [ ]:
sy.gateways

### Domain clients upload some datasets

In [ ]:
input_data = np.array([1, 2, 3])
mock_data = np.array([4, 5, 6])

asset_name = "asset_1"
asset = sy.Asset(name=asset_name, data=input_data, mock=mock_data)
dataset_name = "dataset_1"
dataset = sy.Dataset(name=dataset_name, asset_list=[asset])

dataset_res = domain_client1.upload_dataset(dataset)
dataset_res

In [ ]:
input_data = np.array([7, 8, 9])
mock_data = np.array([10, 11, 12])

asset_name = "asset_2"
asset = sy.Asset(name=asset_name, data=input_data, mock=mock_data)
dataset_name = "dataset_2"
dataset = sy.Dataset(name=dataset_name, asset_list=[asset])

dataset_res = domain_client2.upload_dataset(dataset)
dataset_res

### Test `sy.search` for datasets across domains

In [ ]:
sy.search(name="dataset_1")

### Domain clients create some DS accounts

In [ ]:
domain_client1.register(
    name="data-scientist-1",
    email="test_user1@openmined.org",
    password="0000",
    password_verify="0000",
)
ds_client1 = sy.login(email="test_user1@openmined.org", password="0000", port=9091)
ds_client1

In [ ]:
domain_client2.register(
    name="data-scientist-2",
    email="test_user2@openmined.org",
    password="0000",
    password_verify="0000",
)
ds_client2 = sy.login(email="test_user2@openmined.org", password="0000", port=9092)
ds_client2

In [ ]:
domain_client1.datasets[0].assets[0]

In [ ]:
ds_client2.datasets[0].assets[0]

## 3. Test Proxy Clients

In [ ]:
proxy_domain_client1 = gateway_client.peers[0]
proxy_domain_client1

In [ ]:
proxy_domain_client2 = gateway_client.peers[1]

In [ ]:
type(proxy_domain_client1)

In [ ]:
proxy_domain_client1.datasets[0].assets[0]

In [ ]:
proxy_ds1 = proxy_domain_client1.login(
    email="test_user1@openmined.org",
    password="0000",
)

In [ ]:
proxy_ds1.datasets[0].assets[0]

In [ ]:
proxy_domain_client1.connection.proxy_target_uid

In [ ]:
domain_client1.datasets[1].assets[0]

In [ ]:
ds_client2.datasets[0].assets[0].data

## 4. Remote Code Execution

In [ ]:
asset = proxy_ds1.datasets[0].assets[0]


@sy.syft_function_single_use(asset=asset)
def test_function(asset):
    return asset + 1

In [ ]:
test_function(ds_client1.datasets[0].assets[0])

In [ ]:
proxy_domain_client1.code.submit(test_function)

In [ ]:
domain_client1.code[0]

In [ ]:
z = proxy_domain_client1.code.get_all()

In [ ]:
z[0]

In [ ]:
proxy_ds2 = proxy_domain_client2.login(
    email="test_user2@openmined.org",
    password="0000",
)

In [ ]:
asset = proxy_ds1.datasets[0].assets[0]
asset2 = proxy_ds2.datasets[0].assets[0]

In [ ]:
@sy.syft_function_single_use(asset=asset, asset2=asset2)
def test_function_2(asset, asset2):
    return (asset + 1, asset2 + 1)

In [ ]:
test_function_2(ds_client1.datasets[0].assets[0], ds_client2.datasets[0].assets[0])

In [ ]:
test_function_2.input_policy_init_kwargs.keys()

In [ ]:
proxy_domain_client1.code.submit(test_function_2)

In [ ]:
request_res = proxy_domain_client1.code.request_code_execution(test_function_2)
request_res

In [ ]:
tf2 = domain_client1.code.get_all()[1]
tf2

In [ ]:
tf2(domain_client1.datasets[1].assets[0], domain_client1.datasets[1].assets[0])

In [ ]:
domain_client1.code[1](

In [ ]:
domain_client1.code[1].status

In [ ]:
gateway_client.domains

In [ ]:
zzz = gateway_client.domains[0]
zzz.datasets

In [ ]:
zzz2 = gateway_client.domains[1]
zzz2.datasets

## 5. Veilid Functionalities

In [ ]:
gateway_client.api.services.veilid.generate_dht_key()

In [ ]:
domain_client1.api.services.veilid.generate_dht_key()

In [ ]:
domain_route = domain_client1.api.services.veilid.get_veilid_route()

In [ ]:
domain_route

In [ ]:
gateway_route = gateway_client.api.services.veilid.get_veilid_route()
gateway_route

In [ ]:
gateway_route.dht_key

In [ ]:
domain_route.dht_key

In [ ]:
domain_client1.connect_to_gateway(gateway_client, protocol="veilid")